# Spam Filter using Continuous Naive Bayes

Rodrigo Cedeno Jimenez <br />
Tarea 03 <br />
Machine Learning

### Importamos librerías

In [33]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from sklearn import preprocessing
from random import random

### Importamos base de datos

In [34]:
df = pd.read_csv('Archivos_Ejercicio/spambase.data',header=None)

### Dividimos muestra en Training y Testing. Por regla de dedo Training es 75% y Testing 25%

In [65]:
# Divide data into traning and test
# Es importante notar que se utiliza como Y la muestra que nos indica qué correo es SPAM y cual no, 
# con esto entrenaremos a nuestro sistema
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]], df[df.columns[-1]], train_size = 0.75)

Utilizar sklearn nos evita el normalizar los datos porque ya lo hace por nosotros

### Calculamos probabilidad de que sea SPAM

In [110]:
P_Spam = float(sum(Y_train))/len(Y_train)
P_No_Spam = 1 - P_Spam

### Calcular P(X | SPAM)

In [111]:
mediaS = np.mean(X_train[Y_train == 1], axis = 0)
dsS = np.std (X_train[Y_train == 1], axis = 0)
mediaNS = np.mean(X_train[Y_train == 0], axis = 0)
dsNS = np.std(X_train[Y_train == 0], axis = 0)

### Obtener los valores que son SPAM y los que no son SPAM

In [112]:
# Convierto los valores a arrays porque de lo contrario no puedo calcular los valores que son Spam y los que no.
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [113]:
# Itero en los arrays establecidos para establecer cuales son SPAM y cuales no.
# Con estos ciclos se crean nuevos arrays con ambos valores
Values_Spam = np.array([X_train[i] for i in range(len(X_train)) if Y_train[i]==1])
Values_No_Spam = np.array([X_train[i] for i in range(len(X_train)) if Y_train[i]==0])

### Calcular si es SPAM o no

In [130]:

def GuessIfSpam (x):
    #Se aplican valores logarítmicos para que los valores no se hagan cada vez mas pequeños al multiplicarlos
    Spam = np.log(P_Spam)
    NotSpam= np.log(P_No_Spam)
    Calc_SPAM = 0
    Calc_No_SPAM = 0
    
    for i in range(len(Values_Spam[0])):
        if mediaS[i]!= 0.0 and dsS[i]>0.0:
            Calc_SPAM = Calc_SPAM + np.log(norm(mediaS[i],dsS[i]).pdf(x[i]))

    for i in range(len(Values_No_Spam[1])):
        if mediaNS[i]!= 0.0 and dsNS[i]>0.0:
            Calc_No_SPAM = Calc_No_SPAM + np.log(norm(mediaNS[i],dsNS[i]).pdf(x[i]))

    if (Spam + Calc_SPAM) > (NotSpam + Calc_No_SPAM):
        return 1
    else:
        return 0

In [131]:
Values_No_Spam

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          3.75000000e+00,   9.00000000e+00,   1.50000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          3.08600000e+00,   5.50000000e+01,   1.42000000e+02],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.75000000e+00,   1.50000000e+01,   4.20000000e+01],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   2.40000000e-01, ...,
          2.77600000e+00,   1.21000000e+02,   6.22000000e+02],
       [  0.00000000e+00,   1.93000000e+00,   7.70000000e-01, ...,
          1.04300000e+00,   4.00000000e+00,   9.50000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   5.10000000e-01, ...,
          1.76700000e+00,   7.00000000e+00,   9.90000000e+01]])

In [132]:
Prediction = [GuessIfSpam(x) for x in X_test]

c:\python27\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
c:\python27\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


In [133]:
confusion_matrix(Y_test, Prediction)

array([[501, 186],
       [ 15, 449]], dtype=int64)

### La realidad

In [134]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
Y_Predict = gnb.fit(X_train, Y_train)

In [136]:
confusion_matrix(Y_test,Y_Predict.predict(X_test))

array([[501, 186],
       [ 16, 448]], dtype=int64)

Podemos observar que nuestro modelo predictivo se aproxima muchísimo a la realidad

### Bibliografía

https://appliedmachinelearning.wordpress.com/2017/01/23/email-spam-filter-python-scikit-learn/ <br />
https://www.analyticsvidhya.com/blog/2015/09/naive-bayes-explained/ <br />
http://scikit-learn.org/stable/modules/naive_bayes.html <br />
https://appliedmachinelearning.wordpress.com/2017/05/23/understanding-naive-bayes-classifier-from-scratch-python-code/ <br />